In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


In [3]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, google_search
from google.genai import types
import pandas as pd

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [5]:
# ADK Web UI Helper 
from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers

def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]['base_url']

    try:
        path_parts = baseURL.split('/')
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style='padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;'>
        <div style='font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;'>
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style='font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;'>
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style='margin-top: 10px; padding-left: 20px;'>
                <li style='margin-bottom: 5px;'><strong>Run the next cell</strong> (the one with <code>!adk web --port 8000</code>).</li>
                <li style='margin-bottom: 5px;'>Wait for that cell to show it is "Running".</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style='font-size: 0.9em; color: #555;'>If you click the button too early, you will get a 500 error.</em>
        </div>
        <a href='{url}' target='_blank' style='
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;'>
            Open ADK Web UI ↗
        </a>
    </div>
    """

    display(HTML(styled_html))
    return url_prefix

print("✅ Helper functions defined.")


✅ Helper functions defined.


In [6]:
# Imports and retry config
import os
import time
from typing import Dict, Any, List, Optional
from collections import defaultdict

import pandas as pd

# ADK imports (Kaggle-compatible)
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, google_search

# Gemini retry config
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,
    initial_delay=1,
    exp_base=2,
    max_delay=10,
    jitter=0.3,
    http_status_codes=[408, 429, 500, 502, 503, 504]
)

print("✅ Imports and retry config ready.")


✅ Imports and retry config ready.


In [7]:
# Product catalog
products = [
    {"product_id": 1, "name": "Red Cotton Kurti", "category": "Kurti", "sizes": "S,M,L,XL", "colors": "Red", "price": 799},
    {"product_id": 2, "name": "Blue Rayon Kurti", "category": "Kurti", "sizes": "M,L", "colors": "Blue", "price": 899},
    {"product_id": 3, "name": "Black Skinny Jeans", "category": "Jeans", "sizes": "28,30,32,34", "colors": "Black", "price": 1199},
    {"product_id": 4, "name": "Pink Embroidered Lehenga", "category": "Lehenga", "sizes": "S,M,L", "colors": "Pink", "price": 4999},
    {"product_id": 5, "name": "Banarasi Silk Saree", "category": "Saree", "sizes": "Free", "colors": "Red", "price": 2499},
    {"product_id": 6, "name": "Pink Chiffon Top", "category": "Top", "sizes": "S,M,L", "colors": "Pink", "price": 599},
    {"product_id": 7, "name": "Yellow Summer Dress", "category": "Dress", "sizes": "XS,S,M", "colors": "Yellow", "price": 1299},
]

products_df = pd.DataFrame(products)
PRODUCT_TEXT = products_df.to_string(index=False)
products_df


,product_id,name,category,sizes,colors,price
0,1,Red Cotton Kurti,Kurti,"S,M,L,XL",Red,799
1,2,Blue Rayon Kurti,Kurti,"M,L",Blue,899
2,3,Black Skinny Jeans,Jeans,"28,30,32,34",Black,1199
3,4,Pink Embroidered Lehenga,Lehenga,"S,M,L",Pink,4999
4,5,Banarasi Silk Saree,Saree,Free,Red,2499
5,6,Pink Chiffon Top,Top,"S,M,L",Pink,599
6,7,Yellow Summer Dress,Dress,"XS,S,M",Yellow,1299


In [8]:
# Observability / Metrics
metrics = {
    "total_requests": 0,
    "successful_requests": 0,
    "failed_requests": 0,
    "average_response_time": 0.0,
    "agent_calls": defaultdict(int),
    "tool_calls": defaultdict(int),
    "intent_frequency": defaultdict(int),
    "product_color_queries": defaultdict(int),
    "product_category_queries": defaultdict(int),
    "budget_queries": defaultdict(int),
    "occasion_queries": defaultdict(int),
    "workflow_chain_usage": defaultdict(int)
}

def track_metrics(agent_name: str, tool_name: str, success: bool, response_time: float, intent: Optional[str] = None, color: Optional[str] = None, category: Optional[str] = None, budget: Optional[str] = None, occasion: Optional[str] = None, workflow_chain: Optional[str] = None):
    metrics["total_requests"] += 1
    metrics["agent_calls"][agent_name] += 1
    metrics["tool_calls"][tool_name] += 1
    if success:
        metrics["successful_requests"] += 1
    else:
        metrics["failed_requests"] += 1
    prev_avg = metrics["average_response_time"]
    n = metrics["total_requests"]
    metrics["average_response_time"] = (prev_avg * (n - 1) + response_time) / n
    if intent:
        metrics["intent_frequency"][intent] += 1
    if color:
        metrics["product_color_queries"][color.lower()] += 1
    if category:
        metrics["product_category_queries"][category.lower()] += 1
    if budget:
        metrics["budget_queries"][budget] += 1
    if occasion:
        metrics["occasion_queries"][occasion.lower()] += 1
    if workflow_chain:
        metrics["workflow_chain_usage"][workflow_chain] += 1

def get_metrics() -> Dict[str, Any]:
    return {
        "total_requests": metrics["total_requests"],
        "successful_requests": metrics["successful_requests"],
        "failed_requests": metrics["failed_requests"],
        "success_rate": metrics["successful_requests"] / metrics["total_requests"] if metrics["total_requests"]>0 else 0,
        "average_response_time_seconds": metrics["average_response_time"],
        "agent_calls": dict(metrics["agent_calls"]),
        "tool_calls": dict(metrics["tool_calls"]),
        "intent_frequency": dict(metrics["intent_frequency"]),
        "product_color_queries": dict(metrics["product_color_queries"]),
        "product_category_queries": dict(metrics["product_category_queries"]),
        "budget_queries": dict(metrics["budget_queries"]),
        "occasion_queries": dict(metrics["occasion_queries"]),
        "workflow_chain_usage": dict(metrics["workflow_chain_usage"])
    }

print("✅ Observability module loaded.")


✅ Observability module loaded.


In [9]:
# Evaluation helpers
from typing import List

def evaluate_clothing_response_quality(response_text: str, expected_keywords: Optional[List[str]] = None) -> Dict[str, Any]:
    fashion_markers = ["kurtis","dress","jeans","color","style","size","budget","occasion","wedding","party","recommend","trend"]
    evaluation = {
        "response_length": len(response_text),
        "mentions_fashion_terms": any(m in response_text.lower() for m in fashion_markers),
        "contains_expected_keywords": True,
        "structured_suggestions": any(w in response_text.lower() for w in ["you can also", "additionally", "i recommend", "pair it with"]),
        "quality_score": 0.0
    }
    if expected_keywords:
        evaluation["contains_expected_keywords"] = all(keyword.lower() in response_text.lower() for keyword in expected_keywords)
    score = 0.0
    if evaluation["response_length"] > 60:
        score += 0.3
    if evaluation["mentions_fashion_terms"]:
        score += 0.3
    if evaluation["structured_suggestions"]:
        score += 0.2
    if evaluation["contains_expected_keywords"]:
        score += 0.2
    evaluation["quality_score"] = score
    return evaluation

def evaluate_product_search_result(product_list: List[Dict[str,Any]]) -> Dict[str, Any]:
    if not isinstance(product_list, list):
        return {"valid": False, "error": "Not a list"}
    return {
        "valid": True,
        "results_count": len(product_list),
        "has_name": all("name" in p for p in product_list),
        "has_price": all("price" in p for p in product_list),
        "has_color": all("colors" in p for p in product_list),
        "has_category": all("category" in p for p in product_list)
    }

print("✅ Evaluation helpers ready.")


✅ Evaluation helpers ready.


In [ ]:
!adk create clothing_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

In [11]:
# Single Orchestrator Agent (handles search, size, styling, orders, research, image requests)
# Graceful MCP fallback - since Kaggle doesn’t support MCP image tools
mcp_image_server = None

agent_instruction = f"""
You are a friendly & helpful Clothing Assistant.

You have internal knowledge of the product catalog below:

{PRODUCT_TEXT}

Capabilities:
- Search catalog and return matching items (name, price, sizes, colors)
- Give size advice based on sizes column
- Track orders (mock): always respond "In transit, expected 3–5 days"
- Provide styling suggestions for occasions (wedding, party, office)
- If user asks 'research' or 'latest trends', use the google_search tool
- If user asks for an image and MCP is available, use the MCP Image tool

When answering:
- Always present clear, short results
- When applicable, provide product_id for quick ordering
- If multiple steps are needed (search -> size check -> style), perform them and summarise
"""

# Build tools list safely
tools_list = [google_search]

if mcp_image_server is not None:
    tools_list.append(mcp_image_server)

clothing_agent = Agent(
    name="clothing_assistant",
    model="gemini-2.5-flash-lite",
    instruction=agent_instruction,
    tools=tools_list
)

print("✅ Clothing Assistant Agent created successfully! (single orchestrator)")
print("🔧 Registered tools:", [t.__class__.__name__ for t in tools_list])


✅ Clothing Assistant Agent created successfully! (single orchestrator)
🔧 Registered tools: ['GoogleSearchTool']


In [14]:
# Runner
runner = InMemoryRunner(clothing_agent)
print("🚀 Runner ready!")


🚀 Runner ready!


In [13]:
#  Test calls (run each as cell to see debug traces)
# 1) Simple product search
await runner.run_debug("Show me red kurtis")

# 2) Size help
await runner.run_debug("I am 5'3. What size kurti should I buy?")

# 3) Order tracking
await runner.run_debug("Track order 12345")

# 4) Styling suggestion for wedding
await runner.run_debug("What should I wear for a wedding?")

# 5) Research with google_search
await runner.run_debug("Research the latest kurti trends for 2025")

# 6) Image generation (if MCP available)
if mcp_image_server:
    await runner.run_debug("Generate a preview image of a red kurti with embroidery")



 ### Continue session: debug_session_id

User > Show me red kurtis
clothing_assistant > I found a red cotton kurti for you!

**Red Cotton Kurti**
* Price: 799
* Sizes: S, M, L, XL
* Colors: Red
* Product ID: 1

 ### Continue session: debug_session_id

User > I am 5'3. What size kurti should I buy?
clothing_assistant > I can help with that! To give you the best recommendation, could you please tell me which kurti you are interested in? Knowing the specific product will help me check the available sizes.

 ### Continue session: debug_session_id

User > Track order 12345
clothing_assistant > In transit, expected 3–5 days.

 ### Continue session: debug_session_id

User > What should I wear for a wedding?
clothing_assistant > For a wedding, you have a few stunning options depending on your style and the wedding's formality:

**Lehenga:**
If you're looking for something traditional and grand, the **Pink Embroidered Lehenga** (Product ID: 4) is a beautiful choice. It's available in sizes S, 

In [15]:
# Check metrics and evaluation example
print("Metrics so far:")
print(get_metrics())

# Example evaluation of a sample response
sample_resp = "I recommend a Red Cotton Kurti (₹799). Sizes available: S, M, L, XL. You can pair it with silver earrings."
print(evaluate_clothing_response_quality(sample_resp, expected_keywords=["kurti","sizes","pair"]))


Metrics so far:
{'total_requests': 0, 'successful_requests': 0, 'failed_requests': 0, 'success_rate': 0, 'average_response_time_seconds': 0.0, 'agent_calls': {}, 'tool_calls': {}, 'intent_frequency': {}, 'product_color_queries': {}, 'product_category_queries': {}, 'budget_queries': {}, 'occasion_queries': {}, 'workflow_chain_usage': {}}
{'response_length': 106, 'mentions_fashion_terms': True, 'contains_expected_keywords': True, 'structured_suggestions': True, 'quality_score': 1.0}


In [16]:
# Local catalog utilities (used for deterministic replies in tests)
def local_search_products(query: str = None, color: str = None, category: str = None, max_price: Optional[float] = None):
    results = products_df.to_dict(orient="records")
    if query:
        q = query.lower()
        results = [p for p in results if q in p["name"].lower() or q in p["category"].lower()]
    if color:
        results = [p for p in results if p["colors"].lower() == color.lower()]
    if category:
        results = [p for p in results if p["category"].lower() == category.lower()]
    if max_price:
        results = [p for p in results if p["price"] <= max_price]
    return results

print("✅ Local catalog utilities ready.")


✅ Local catalog utilities ready.


In [17]:
# 📌 Tool: Product Search Tool for Orchestrator Agent
def search_products_tool(query: str = None, color: str = None, category: str = None, max_price: Optional[float] = None):
    """
    Search clothing catalog by:
    - query (name/category contains this text)
    - color match
    - category match
    - price under X
    """
    results = local_search_products(query=query, color=color, category=category, max_price=max_price)
    
    if not results:
        return {"message": "No matching items found!"}

    # Format for response
    return {
        "count": len(results),
        "items": [
            {
                "product_id": r["id"],
                "name": r["name"],
                "category": r["category"],
                "price": r["price"],
                "available_sizes": r["sizes"],
                "color": r["colors"]
            }
            for r in results
        ]
    }

print("🔧 Product Search Tool added!")


🔧 Product Search Tool added!


In [20]:
# Build tools list with local product search added
tools_list = [google_search, search_products_tool]  # Added new tool!

# MCP is disabled in Kaggle, only append if available
if mcp_image_server is not None:
    tools_list.append(mcp_image_server)


In [ ]:
# CELL 3 — Start ADK Web UI (run this cell and leave it running)
url_prefix = get_adk_proxy_url()

In [ ]:
!adk web --url_prefix {url_prefix}